# Importing the packages

In [ ]:
!pip install rioxarray stackstac pystac-client planetary-computer odc-stac rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 12.3 MB/s eta 0:00:00


In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import jet,RdYlGn

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

In [ ]:
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

In [ ]:
# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

## Loading the key bands from Lansdat-8 satellite with the help of microsoft planetary computer

In [ ]:
# We will use a period of 3 months to search for data
time_window = "2021-06-01/2021-09-30"

In [ ]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 10},"platform": {"in": ["landsat-8"]}},
)

In [ ]:
items = list(search.get_items())
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [ ]:
unique_bands = set()

# Iterate over items and extract the band names
for item in search.items():
    for asset_key in item.assets:
        # Add the asset name (band name) to the set
        unique_bands.add(asset_key)

# Print the unique band names
print(unique_bands)

{'mtl.json', 'atran', 'ang', 'swir22', 'nir08', 'drad', 'mtl.txt', 'rendered_preview', 'coastal', 'urad', 'mtl.xml', 'qa_aerosol', 'cdist', 'qa_radsat', 'tilejson', 'qa_pixel', 'swir16', 'emis', 'green', 'red', 'qa', 'blue', 'emsd', 'trad', 'lwir11'}


In [ ]:
resolution = 30  # meters per pixel (native resolution of Landsat-8)
crs = "EPSG:32618"  # UTM Zone 18N (meters)

data = stac_load(
    items,
    bands=["red", "green", "blue", "nir08", "swir16", "swir22", "lwir11", "emis"],
    crs=crs,  # Projected CRS (UTM)
    resolution=resolution,  # In meters
    chunks={"x": 2048, "y": 2048},
    dtype="float32",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

In [ ]:
display(data)

<xarray.Dataset> Size: 27MB
Dimensions:      (y: 487, x: 428, time: 4)
Coordinates:
  * y            (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x            (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref  int32 4B 32618
  * time         (time) datetime64[ns] 32B 2021-06-07T15:39:15.904901 ... 202...
Data variables:
    red          (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    green        (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    blue         (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    nir08        (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    swir16       (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    swir22       (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    lwir11       (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>
    emis         (time, y, x) float32 3MB dask.array<chunksize=(1, 487, 428), meta=np.ndarray>

# Scaling key bands

In [ ]:
# Apply scaling to the RGB, NIR, SWIR bands
scale1 = 0.0000275
offset1 = -0.2

# Apply scaling to the Surface Temperature band
scale2 = 0.00341802
offset2 = 149.0
kelvin_celsius = 273.15  # Convert from Kelvin to Celsius

scale_emis = 0.0001  # Convert from integer range (0–10000) to (0–1) if needed

# Separate the bands for scaling
rgb_nir_bands = ["red", "green", "blue", "nir08", "swir16", "swir22"]
surface_temp_band = ["lwir11"]
emis_band = ["emis"]

# Apply scaling to the relevant bands
for band in rgb_nir_bands:
    data[band] = data[band].astype(float) * scale1 + offset1

# Apply scaling for the surface temperature band (lwir11)
for band in surface_temp_band:
    data[band] = data[band].astype(float) * scale2 + offset2 - kelvin_celsius

# Apply scaling for the emissivity band (emis)
for band in emis_band:
    data[band] = data[band].astype(float) * scale_emis

# Computing the median of bands across the time period

In [ ]:
# Compute median composite across time dimension
data_median = data.median(dim="time", skipna=True)

print("Median Composite Computed")
display(data_median)

Median Composite Computed


<xarray.Dataset> Size: 13MB
Dimensions:      (y: 487, x: 428)
Coordinates:
  * y            (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x            (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref  int32 4B 32618
Data variables:
    red          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16       (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22       (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    lwir11       (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    emis         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# Feature engineering

# Calculating LST (Land surface temp)

In [ ]:
# Constants from Landsat-8 metadata
K1 = 774.89   # Thermal constant 1 for Landsat-8 Band 11
K2 = 1321.08  # Thermal constant 2 for Landsat-8 Band 11
LAMBDA = 10.9e-6  # Wavelength in meters (for Band 11)
RHO = 1.438e-2  # Constant for LST equation

# Extract already scaled bands
lwir11 = data_median["lwir11"]  # Already converted to Celsius
emis = data_median["emis"]  # Already scaled (0–1 range)

# Step 1: Compute Brightness Temperature (BT)
BT = K2 / (np.log((K1 / lwir11) + 1))

# Step 2: Compute LST using Emissivity Correction
data_median['LST'] = BT / (1 + (LAMBDA * BT / RHO) * np.log(emis))

# NDVI (Normalized Difference Vegetation Index)

In [ ]:
data_median["landsat_NDVI"] = (data_median["nir08"] - data_median["red"]) / (data_median["nir08"] + data_median["red"])

print("NDVI Computed")
display(data_median)

NDVI Computed


<xarray.Dataset> Size: 17MB
Dimensions:       (y: 487, x: 428)
Coordinates:
  * y             (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x             (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref   int32 4B 32618
Data variables:
    red           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    lwir11        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    emis          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    LST           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# NDBI (Normalized Difference Built-up Index)

In [ ]:
data_median["landsat_NDBI"] = (data_median["swir16"] - data_median["nir08"]) / (data_median["swir16"] + data_median["nir08"])

print("NDBI Computed")
display(data_median)

NDBI Computed


<xarray.Dataset> Size: 18MB
Dimensions:       (y: 487, x: 428)
Coordinates:
  * y             (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x             (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref   int32 4B 32618
Data variables:
    red           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    lwir11        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    emis          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    LST           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDBI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# NDWI (Normalized Difference Water Index)

In [ ]:
data_median["landsat_NDWI"] = (data_median["green"] - data_median["nir08"]) / (data_median["green"] + data_median["nir08"])
print("NDWI Computed")
display(data_median)

NDWI Computed


<xarray.Dataset> Size: 20MB
Dimensions:       (y: 487, x: 428)
Coordinates:
  * y             (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x             (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref   int32 4B 32618
Data variables:
    red           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    lwir11        (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    emis          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    LST           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDBI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDWI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# MNDWI (Modified Normalized Difference Water Index)

In [ ]:
data_median["landsat_MNDWI"] = (data_median["green"] - data_median["swir16"]) / (data_median["green"] + data_median["swir16"])
print("MNDWI Computed")
display(data_median)

MNDWI Computed


<xarray.Dataset> Size: 22MB
Dimensions:        (y: 487, x: 428)
Coordinates:
  * y              (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x              (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref    int32 4B 32618
Data variables: (12/13)
    red            (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    ...             ...
    emis           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    LST            (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDBI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDWI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_MNDWI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# SAVI (Soil-Adjusted Vegetation Index)

In [ ]:
L = 0.5
data_median["landsat_SAVI"] = ((data_median["nir08"] - data_median["red"]) / (data_median["nir08"] + data_median["red"] + L)) * (1 + L)
print("SAVI Computed")
display(data_median)

SAVI Computed


<xarray.Dataset> Size: 23MB
Dimensions:        (y: 487, x: 428)
Coordinates:
  * y              (y) float64 4kB 4.526e+06 4.526e+06 ... 4.512e+06 4.511e+06
  * x              (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05 5.962e+05
    spatial_ref    int32 4B 32618
Data variables: (12/14)
    red            (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue           (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08          (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22         (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    ...             ...
    LST            (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDBI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDWI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_MNDWI  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_SAVI   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# Other important features

In [ ]:
data_median['TVX'] = data_median['LST']/data_median['landsat_SAVI']

In [ ]:
data_median['landsat_NDBI*LST'] = data_median['landsat_NDBI'] * data_median['LST']

In [ ]:
data_median['landsat_NDVI*LST'] = (1 - data_median['landsat_NDVI']) * data_median['LST']

In [ ]:
data_median["landsat_Albedo"] = (data_median["red"] + data_median["green"] + data_median["blue"]) / 3

print("Albedo Computed")
display(data_median)

Albedo Computed


<xarray.Dataset> Size: 30MB
Dimensions:           (y: 487, x: 428)
Coordinates:
  * y                 (y) float64 4kB 4.526e+06 4.526e+06 ... 4.511e+06
  * x                 (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05
    spatial_ref       int32 4B 32618
Data variables: (12/18)
    red               (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green             (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue              (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08             (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16            (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22            (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    ...                ...
    landsat_MNDWI     (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_SAVI      (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    TVX               (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDBI*LST  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI*LST  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_Albedo    (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

In [ ]:
data_median["landsat_Brightness"] = (0.3037 * data_median["blue"] + 0.2793 * data_median["green"] + 0.4743 * data_median["red"] +
                      0.5585 * data_median["nir08"] + 0.5082 * data_median["swir16"] + 0.1863 * data_median["swir22"])
data_median["landsat_Greenness"] = (-0.2848 * data_median["blue"] - 0.2435 * data_median["green"] + 0.5436 * data_median["red"] +
                     0.7243 * data_median["nir08"] + 0.0840 * data_median["swir16"] - 0.1800 * data_median["swir22"])
data_median["landsat_Wetness"] = (0.1509 * data_median["blue"] + 0.1973 * data_median["green"] + 0.3279 * data_median["red"] +
                   0.3397 * data_median["nir08"] - 0.1182 * data_median["swir16"] - 0.1190 * data_median["swir22"])


In [ ]:
display(data_median)

<xarray.Dataset> Size: 35MB
Dimensions:             (y: 487, x: 428)
Coordinates:
  * y                   (y) float64 4kB 4.526e+06 4.526e+06 ... 4.511e+06
  * x                   (x) float64 3kB 5.834e+05 5.835e+05 ... 5.962e+05
    spatial_ref         int32 4B 32618
Data variables: (12/21)
    red                 (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    green               (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    blue                (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    nir08               (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir16              (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    swir22              (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    ...                  ...
    landsat_NDBI*LST    (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_NDVI*LST    (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_Albedo      (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_Brightness  (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_Greenness   (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>
    landsat_Wetness     (y, x) float64 2MB dask.array<chunksize=(487, 428), meta=np.ndarray>

# Saving the data into Geotiff file

In [ ]:
filename = "Landsat_LST.tiff"

# Ensure CRS is correctly assigned
data_median = data_median.rio.write_crs("EPSG:32618", inplace=True)

# Compute transformation matrix for georeferencing
height = data_median.dims["y"]
width = data_median.dims["x"]
gt = rasterio.transform.from_bounds(lower_left[1], lower_left[0], upper_right[1], upper_right[0], width, height)
data_median.rio.write_transform(transform=gt, inplace=True)

# Save the entire median composite with all computed bands
with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height,
                   crs='EPSG:32618', transform=gt, count=len(data_median.data_vars),
                   compress='lzw', dtype='float64') as dst:

    for i, band in enumerate(data_median.data_vars):  # Iterate through all available bands
        dst.write(data_median[band].values, i+1)

print(f"GeoTIFF saved successfully as: {filename}")

GeoTIFF saved successfully as: Landsat_LST.tiff


# uploading the Geotiff file into drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Define the Google Drive path where you want to save the file
drive_path = "/content/drive/My Drive/Colab Notebooks/Landsat_LST.tiff"

# Save the file to Google Drive
with rasterio.open(drive_path, 'w', driver='GTiff', width=width, height=height,
                   crs='EPSG:32618', transform=gt, count=len(data_median.data_vars),
                   compress='lzw', dtype='float64') as dst:
    for i, band in enumerate(data_median.data_vars):  # Iterate through all available bands
        dst.write(data_median[band].values, i+1)

print(f"GeoTIFF saved successfully at: {drive_path}")

GeoTIFF saved successfully at: /content/drive/My Drive/Colab Notebooks/Landsat_LST.tiff
